# Debug Runt Transform
template for testing/debugging dffd

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname
    
def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('runt_debug_dffd.ipynb', 'mutate' +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit, vectorize
from dask import delayed, dataframe as dd

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)

from common_util import MUTATE_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ
from common_util import load_json, dump_json, df_null_rows, remove_dups_list, list_get_dict, is_empty_df, search_df, benchmark
from data.data_api import DataAPI
from mutate.common import TRANSFORMS_DIR, get_graphs, get_transforms
from mutate.runt import process_transform

DEBUG:matplotlib:$HOME=/home/kev
DEBUG:matplotlib:CONFIGDIR=/home/kev/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /home/kev/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /home/kev/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.1.1
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'types', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib.util', 'impo

DEBUG:matplotlib:CACHEDIR=/home/kev/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/kev/.cache/matplotlib/fontlist-v310.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
CRITICAL:root:script location: /home/kev/crunch/mutate/runt_debug_dffd.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## General

### Init DataAPI, Set Logging Level, and Load Transforms

In [2]:
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
transforms = get_transforms()

## Transform Debug

### Set Transform
Fill it in (default is daily ohlca)

In [3]:
trf_name = 'dwrxmx'
step = transforms[trf_name]
display(step)

{'meta': {'rec_fmt': 'dwrx{rank_type}(-1)',
  'var_fmt': 'grid',
  'res_freq': 'D',
  'info': 'daily window rank expanding min max scaled (identical to dwrmx but with num_periods fixed to -1)'},
 'fn': {'ser_fn': 'wrx', 'df_fn': 'rut', 'col_fn': None, 'freq': None},
 'axe': {'rm': None,
  'src': [['dffd', 'dffd_trmi3_news_nonbuzz'],
   ['dffd', 'dffd_trmi3_social_nonbuzz'],
   ['ddiff', 'ddiff_trmi3_news_nonbuzz'],
   ['ddiff', 'ddiff_trmi3_social_nonbuzz']],
  'dst': ['dwrxmx', 'dwrxmx']},
 'var': {'rank_type': ['mx']}}

In [4]:
display(step)

{'meta': {'rec_fmt': 'dwrx{rank_type}(-1)',
  'var_fmt': 'grid',
  'res_freq': 'D',
  'info': 'daily window rank expanding min max scaled (identical to dwrmx but with num_periods fixed to -1)'},
 'fn': {'ser_fn': 'wrx', 'df_fn': 'rut', 'col_fn': None, 'freq': None},
 'axe': {'rm': None,
  'src': [['dffd', 'dffd_trmi3_news_nonbuzz'],
   ['dffd', 'dffd_trmi3_social_nonbuzz'],
   ['ddiff', 'ddiff_trmi3_news_nonbuzz'],
   ['ddiff', 'ddiff_trmi3_social_nonbuzz']],
  'dst': ['dwrxmx', 'dwrxmx']},
 'var': {'rank_type': ['mx']}}

### Test Section

In [5]:
runt_rcs, runt_dfs = process_transform(step, dump=False)

/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


INFO:root:data: ['dow_jones', 'dffd', 'dffd_trmi3_news_nonbuzz', 'trmi3_etf3_news_forecast_dc_dffd', 'trmi3_etf3_news_forecast_dc_dffd(0.200000,0.010000)']
DEBUG:root:pre-runt:                            etf3_timeUrgency_N_close  etf3_uncertainty_N_close  \
id                                                                              
2009-01-11 00:00:00+00:00                  0.007274                  0.006580   
2009-01-12 00:00:00+00:00                  0.006745                  0.008768   
2009-01-13 00:00:00+00:00                  0.005477                  0.007359   
2009-01-14 00:00:00+00:00                  0.007340                  0.009427   
2009-01-15 00:00:00+00:00                  0.004629                  0.011236   
...                                             ...                       ...   
2017-12-27 00:00:00+00:00                  0.007125                  0.006100   
2017-12-28 00:00:00+00:00                  0.009420                  0.007504   
2017-12-29 00

INFO:root:data: ['dow_jones', 'dffd', 'dffd_trmi3_news_nonbuzz', 'trmi3_etf3_news_forecast_dc_dffd', 'trmi3_etf3_news_forecast_dc_dffd(0.400000,0.010000)']
DEBUG:root:pre-runt:                            etf3_timeUrgency_N_close  etf3_uncertainty_N_close  \
id                                                                              
2009-01-11 00:00:00+00:00                  0.003932                  0.002484   
2009-01-12 00:00:00+00:00                  0.003184                  0.004944   
2009-01-13 00:00:00+00:00                  0.001953                  0.003203   
2009-01-14 00:00:00+00:00                  0.004079                  0.005446   
2009-01-15 00:00:00+00:00                  0.001102                  0.006903   
...                                             ...                       ...   
2017-12-27 00:00:00+00:00                  0.000306                  0.001930   
2017-12-28 00:00:00+00:00                  0.003798                  0.003590   
2017-12-29 00

INFO:root:data: ['dow_jones', 'dffd', 'dffd_trmi3_news_nonbuzz', 'trmi3_etf3_news_forecast_dc_dffd', 'trmi3_etf3_news_forecast_dc_dffd(0.600000,0.010000)']
DEBUG:root:pre-runt:                            etf3_timeUrgency_N_close  etf3_uncertainty_N_close  \
id                                                                              
2009-01-09 00:00:00+00:00                 -0.002139                  0.001780   
2009-01-10 00:00:00+00:00                  0.002371                  0.002421   
2009-01-11 00:00:00+00:00                  0.002428                  0.000470   
2009-01-12 00:00:00+00:00                  0.001429                  0.003240   
2009-01-13 00:00:00+00:00                  0.000354                  0.001099   
...                                             ...                       ...   
2017-12-27 00:00:00+00:00                 -0.002948                  0.000203   
2017-12-28 00:00:00+00:00                  0.001632                  0.002006   
2017-12-29 00

INFO:root:data: ['dow_jones', 'dffd', 'dffd_trmi3_news_nonbuzz', 'trmi3_etf3_news_forecast_dc_dffd', 'trmi3_etf3_news_forecast_dc_dffd(0.800000,0.010000)']
DEBUG:root:pre-runt:                            etf3_timeUrgency_N_close  etf3_uncertainty_N_close  \
id                                                                              
2009-01-06 00:00:00+00:00                  0.007017                  0.002288   
2009-01-07 00:00:00+00:00                 -0.001202                 -0.005275   
2009-01-08 00:00:00+00:00                 -0.001426                 -0.000592   
2009-01-09 00:00:00+00:00                 -0.002695                  0.001313   
2009-01-10 00:00:00+00:00                  0.002404                  0.001643   
...                                             ...                       ...   
2017-12-27 00:00:00+00:00                 -0.004358                 -0.000192   
2017-12-28 00:00:00+00:00                  0.001227                  0.001599   
2017-12-29 00

...

DEBUG:root:variant: {'rank_type': 'mx'}
DEBUG:root:subvar: {'rank_type': 'mx'}
DEBUG:root:mutate_type: dwrxmx(-1)
DEBUG:root:post-runt:                            mkt3_stockIndexSentiment_N_close  \
id                                                            
2009-01-10 00:00:00+00:00                         -1.000000   
2009-01-11 00:00:00+00:00                          1.000000   
2009-01-12 00:00:00+00:00                         -1.000000   
2009-01-13 00:00:00+00:00                          1.000000   
2009-01-14 00:00:00+00:00                          0.385478   
...                                                     ...   
2017-12-27 00:00:00+00:00                          0.342635   
2017-12-28 00:00:00+00:00                          0.528601   
2017-12-29 00:00:00+00:00                          0.129231   
2017-12-30 00:00:00+00:00                         -0.046248   
2017-12-31 00:00:00+00:00                          0.198287   

                           mkt3_stockIndexOp

INFO:root:data: ['dow_jones', 'dffd', 'dffd_trmi3_news_nonbuzz', 'trmi3_mkt3_news_stock_dc_dffd', 'trmi3_mkt3_news_stock_dc_dffd(0.800000,0.010000)']
DEBUG:root:pre-runt:                            mkt3_stockIndexSentiment_N_close  \
id                                                            
2009-01-06 00:00:00+00:00                         -0.148675   
2009-01-07 00:00:00+00:00                         -0.040225   
2009-01-08 00:00:00+00:00                         -0.222807   
2009-01-09 00:00:00+00:00                         -0.006345   
2009-01-10 00:00:00+00:00                         -0.078360   
...                                                     ...   
2017-12-27 00:00:00+00:00                          0.196717   
2017-12-28 00:00:00+00:00                          0.208473   
2017-12-29 00:00:00+00:00                          0.003724   
2017-12-30 00:00:00+00:00                         -0.051676   
2017-12-31 00:00:00+00:00                          0.068412   

         

DEBUG:root:variant: {'rank_type': 'mx'}
DEBUG:root:subvar: {'rank_type': 'mx'}
DEBUG:root:mutate_type: dwrxmx(-1)
DEBUG:root:post-runt:                            mkt3_stockIndexSentiment_N_close  \
id                                                            
2009-01-07 00:00:00+00:00                          1.000000   
2009-01-08 00:00:00+00:00                         -1.000000   
2009-01-09 00:00:00+00:00                          1.000000   
2009-01-10 00:00:00+00:00                          0.334623   
2009-01-11 00:00:00+00:00                          1.000000   
...                                                     ...   
2017-12-27 00:00:00+00:00                          0.501365   
2017-12-28 00:00:00+00:00                          0.528823   
2017-12-29 00:00:00+00:00                          0.050601   
2017-12-30 00:00:00+00:00                         -0.078796   
2017-12-31 00:00:00+00:00                          0.201689   

                           mkt3_stockIndexOp

INFO:root:data: ['nasdaq_100', 'dffd', 'dffd_trmi3_news_nonbuzz', 'trmi3_etf3_news_forecast_dc_dffd', 'trmi3_etf3_news_forecast_dc_dffd(0.200000,0.010000)']
DEBUG:root:pre-runt:                            etf3_timeUrgency_N_close  etf3_uncertainty_N_close  \
id                                                                              
2009-01-11 00:00:00+00:00                  0.007853                  0.021946   
2009-01-12 00:00:00+00:00                  0.011063                  0.012685   
2009-01-13 00:00:00+00:00                  0.007134                  0.006732   
2009-01-14 00:00:00+00:00                  0.007078                  0.007893   
2009-01-15 00:00:00+00:00                  0.006247                  0.011369   
...                                             ...                       ...   
2017-12-27 00:00:00+00:00                  0.007408                  0.008073   
2017-12-28 00:00:00+00:00                  0.011449                  0.011779   
2017-12-29 0

KeyboardInterrupt: 

In [5]:
src_data = DataAPI.axe_load(['dc', 'dc'], lazy=False)

/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [6]:
key = ['dow_jones', 'dc', 'dc', 'trmi3_etf3_news_forecast_dc', 'trmi3_etf3_news_forecast_dc']
src_df = src_data[1][key]

In [24]:
src_df.dropna()

,etf3_timeUrgency_N_close,etf3_uncertainty_N_close,etf3_violence_N_close,etf3_emotionVsFact_N_close,etf3_marketRisk_N_close,etf3_longShort_N_close,etf3_longShortForecast_N_close,etf3_priceDirection_N_close,etf3_priceForecast_N_close,etf3_volatility_N_close,etf3_analystRating_N_close
id,,,,,,,,,,,
2009-01-01 00:00:00+00:00,0.015087,0.013831,0.022427,0.554213,-0.013300,0.005389,0.001603,-0.006954,0.000135,0.028659,0.000000
2009-01-02 00:00:00+00:00,0.013575,0.015701,0.026568,0.540200,-0.011233,0.011994,0.000545,-0.002072,-0.000109,0.027277,0.000491
2009-01-03 00:00:00+00:00,0.013521,0.017223,0.026657,0.550871,-0.015350,0.007377,0.002797,0.008255,-0.000329,0.024266,0.000165
2009-01-04 00:00:00+00:00,0.025178,0.014960,0.027515,0.586339,-0.012255,0.005276,0.002204,-0.001269,0.000334,0.028076,0.001469
2009-01-05 00:00:00+00:00,0.009991,0.020272,0.021948,0.547713,-0.002126,0.003841,-0.000199,0.000870,0.000824,0.021211,-0.003823
...,...,...,...,...,...,...,...,...,...,...,...
2017-12-27 00:00:00+00:00,0.018786,0.013730,0.023191,0.553091,-0.007956,0.007622,0.002347,-0.001477,0.000154,0.025982,0.001976
2017-12-28 00:00:00+00:00,0.019826,0.014759,0.035490,0.507919,-0.005872,0.001043,0.000826,0.006218,0.000718,0.026806,0.002548
2017-12-29 00:00:00+00:00,0.020707,0.016212,0.025927,0.548541,-0.007693,0.000237,0.000316,-0.002750,-0.001012,0.034785,0.005349


In [22]:
df_null_rows(src_df).fillna(method='ffill')

,etf3_timeUrgency_N_close,etf3_uncertainty_N_close,etf3_violence_N_close,etf3_emotionVsFact_N_close,etf3_marketRisk_N_close,etf3_longShort_N_close,etf3_longShortForecast_N_close,etf3_priceDirection_N_close,etf3_priceForecast_N_close,etf3_volatility_N_close,etf3_analystRating_N_close
id,,,,,,,,,,,
2010-12-26 00:00:00+00:00,0.025191,0.034655,0.037160,0.552859,-0.012066,0.003758,0.002505,0.000557,NaN,0.032650,0.001392
2012-09-02 00:00:00+00:00,-0.027271,0.015340,0.057099,0.550878,-0.033237,0.000000,-0.001704,-0.003409,NaN,0.045679,0.001392
2012-09-03 00:00:00+00:00,0.032926,0.028705,0.023076,0.453481,-0.025300,-0.028142,0.000563,-0.000563,-0.000563,0.032532,0.001392
